In [10]:
import pandas as pd
from joblib import Parallel, delayed
import os

In [2]:
os.chdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\notmeaningfultransfer")

In [7]:
files = os.listdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\notmeaningfultransfer")

In [4]:
def find_permnos(file, crosswalk):
    patents  = pd.read_csv(file)
    columns_new = []
    columns = patents.columns
    
    intersected_company_names = set(patents['assignor'].dropna().values).intersection(set(crosswalk['company'].values))
    temp = crosswalk[crosswalk['company'].isin(intersected_company_names)]
    company_permno_dict = dict(zip(temp['company'].values, temp['permno'].values))
    patents['sellerpermno'] = patents['assignor'].map(company_permno_dict)
    columns_new.extend(['assignor', 'sellerpermno'])
    
    i = 0
    while 'name_' + str(i)  in columns:
        buyer = 'name_' + str(i)
        intersected_company_names = set(patents[buyer].dropna().values).intersection(set(crosswalk['company'].values))
        temp = crosswalk[crosswalk['company'].isin(intersected_company_names)]
        company_permno_dict = dict(zip(temp['company'].values, temp['permno'].values))
        patents['buyerpermno_' + str(i)] = patents[buyer].map(company_permno_dict)
        columns_new.extend([buyer, 'buyerpermno_' + str(i)])
        i = i + 1    
    columns_list = ['notmeaningful', 'patent_ids', 'recorded_date', 'reel-no', 'frame-no', 'total_number_assignees']
    columns_list.extend(columns_new)
    columns_list.extend(list(set(columns)-set(columns_list)))
    patents = patents[columns_list]
    patents.to_csv(file[:-4] +"_final.csv", index = False)
    #drop empty columns which we cannot find permnos for multiple assignees
    return patents

In [5]:
crosswalk = pd.read_csv("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\identifycompanypermno\\intermediate_output\\permno_processedallcompanyames_crosswalk.csv")

In [8]:
files

['2013_01_notmeaningfultransfer.csv',
 '2013_02_notmeaningfultransfer.csv',
 '2014_01_notmeaningfultransfer.csv',
 '2014_02_notmeaningfultransfer.csv',
 '2015_notmeaningfultransfer.csv',
 'ad20121231-01_notmeaningfultransfer.csv',
 'ad20121231-02_notmeaningfultransfer.csv',
 'ad20121231-03_notmeaningfultransfer.csv',
 'ad20121231-04_notmeaningfultransfer.csv',
 'ad20121231-05_notmeaningfultransfer.csv',
 'ad20121231-06_notmeaningfultransfer.csv',
 'ad20121231-07_notmeaningfultransfer.csv',
 'ad20121231-08_notmeaningfultransfer.csv',
 'ad20121231-09_notmeaningfultransfer.csv',
 'ad20121231-10_notmeaningfultransfer.csv',
 'ad20121231-11_notmeaningfultransfer.csv',
 'ad20121231-12_notmeaningfultransfer.csv']

In [ ]:
#notmeanigful patent transfers files
#If the transfer is not meaningful, same permno

In [11]:
#multiprocessing for files
with Parallel(n_jobs=-1, verbose=1, prefer="threads") as parallel:
    parallel(delayed(find_permnos)(file, crosswalk) for file in files)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
C:\Users\fjying\anaconda3\lib\site-packages\joblib\_parallel_backends.py:608: DtypeWarning: Columns (1,16,18,19,20,21,24,26,29,30,31,33,34,35,36,37,38,39,40,42,43,45,49,51,52,54) have mixed types.Specify dtype option on import or set low_memory=False.
  return self.func(*args, **kwargs)
C:\Users\fjying\anaconda3\lib\site-packages\joblib\_parallel_backends.py:608: DtypeWarning: Columns (8,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,78,79) have mixed types.Specify dtype option on import or set low_memory=False.
  return self.func(*args, **kwargs)
C:\Users\fjying\anaconda3\lib\site-packages\joblib\_parallel_backends.py:608: DtypeWarning: Columns (1,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,56,57,58,59,60,61,62,

[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:  9.5min finished
